In [1]:
import numpy as np
import pandas as pd
import ast
from sentence_transformers import util
import matplotlib.pyplot as plt
from itertools import chain


In [2]:
import time
import os
import openai
import base64
import requests
import pandas as pd
from io import StringIO

#Here your API key
#openai.api_key = ...



### Test OpenAi o3-mini-2025-01-31

In [3]:
def talk2openai(prompt):
    messages=[]
    
    #openai headers
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {openai.api_key}"
    }
    
    #preparing the messages with the image and the prompt

    #print()
    #print ("=== prompt")
    #print (prompt)
    
    messages.append(
          {
            "role": "user",
            "content": [
              {"type": "text", "text": prompt
                  },
    
            ],
          }
        )
     
    response = openai.chat.completions.create(
        model="o1-mini",
        #model="gpt-4o-mini",
        messages=messages
      )
    
    answer = response.choices[0].message.content
    return answer


In [4]:
prompt = "hi"
print (talk2openai(prompt))

Hello! 😊 How can I help you today?


### Read dataset and generate prompts

In [5]:
jokes = pd.read_csv('jokes_base.csv', encoding='utf-8')


In [10]:
jokes.iloc[0].keywords


'[["waitress\'s", \'dish it out\'], [\'\'], [\'\'], [\'waitress\', \'daughter\', \'dish it out\'], [\'\']]'

In [16]:
jokes.iloc[1].sentence_2


'He was in a lot of pain while he was at the hospital.'

In [174]:
jokes

,ID,explanation,keywords,joke,funniness_average,filtered_funniness_average,sentence_1,word_1,sentence_2,word_2
0,hom_657,['the phrase dish it out means to criticizie a...,"[[""waitress's"", 'dish it out'], [''], [''], ['...",She was only a Waitress ' s daughter but she s...,0.6,1.000000,A waitress serves food to customers on a dish.,dish,He can dish it out but he can't take it when o...,dish
1,het_1635,"['', 'This is a play on words. The words ""pane...","[[''], ['glazier', 'window', 'feel your pane',...",When the glazier was sent to the hospital room...,1.2,2.000000,The glazier installs a pane in the window.,pane,He was in a lot of pain while he was at the ho...,pain
2,hom_1510,"['sandpaper is normally rough on one side ', '...","[['inventor', 'sandpaper', 'rough'], ['invento...",The inventor of sandpaper had a rough time of ...,1.6,1.600000,I went through a rough time when I lost my job.,rough,The material was quite rough.,rough
3,hom_955,"['', '', '', 'A contortionist is someone who c...","[[''], [''], [''], ['contortionists', 'die', '...","OLD CONTORTIONISTS never die , they just meet ...",0.2,1.000000,NaN,NaN,NaN,NaN
4,hom_1505,"['', 'This is a play on words. The word ""party...","[[''], ['whose place to have a party at', 'par...",When you can ' t decide with your friends whos...,0.4,0.666667,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1890,hom_2076,"['', ""This is a pun on the phrase 'stroke of g...","[[''], ['Da Vinci', 'stroke', 'genius'], ['Leo...",Leonardo Da Vinci could use a paint brush to m...,0.4,0.500000,NaN,NaN,NaN,NaN
1891,hom_1437,['driven can mean to drive a car or it can me ...,"[['cars', 'driven'], ['cars', 'driven', 'eden'...",The earliest mention of cars is in the Bible w...,1.0,1.250000,NaN,NaN,NaN,NaN
1892,het_1530,"['son sounds like sun ', ""This is a pun on 'su...","[['doting', 'son'], ['parents', 'son', 'worshi...",Some doting parents are son worshipers .,0.8,1.000000,NaN,NaN,NaN,NaN
1893,het_100,"['lion sounds like line ', ""This is a pun on t...","[['zoo', 'lion'], ['phone', 'zoo', 'lion'], ['...",I phoned the zoo but the lion was busy .,1.6,1.600000,NaN,NaN,NaN,NaN


### Check contents

In [142]:

for i in range(2):
    print ("===")
    print("ID:",jokes['ID'][i])
    print("JOKE:",jokes['joke'][i])
    
    explanations = ast.literal_eval(jokes['explanation'][i])
    explanations = [element for element in explanations if element]

    if len(explanations) > 0:
        print ("\nEXPLANATIONS:")
        for k in range(len(explanations)):
            if explanations[k] != '': print(explanations[k])

    keywords = ast.literal_eval(jokes['keywords'][i])
    keywords = list(chain.from_iterable(keywords))
    keywords = [element for element in keywords if element]
    if len(keywords) > 0:
        print ("\nKEYWORDS:")
        for k in range(len(keywords)):
            if keywords[k] != "": print(keywords[k], end=", ")
    
    print (end='\n')
    print (end='\n')

    

===
ID: hom_657
JOKE: She was only a Waitress ' s daughter but she sure could dish it out .

EXPLANATIONS:
the phrase dish it out means to criticizie and waitresses normally carry dishes to tables 
A waitress serves food to customers on dishes. To "dish it out" means to be able to criticize.

KEYWORDS:
waitress's, dish it out, waitress, daughter, dish it out, 

===
ID: het_1635
JOKE: When the glazier was sent to the hospital room to check the cracked window , he told the patient in the body cast , ' ' I ' ve come to feel your pane . ' '

EXPLANATIONS:
This is a play on words. The words "pane" is a sheet of glass that a glazier would fit to a window. However, "pain" means physical discomfort often felt by a hospital patient.
A glazier is someone who fits glass into windows. A pane is a single sheet of glass. Instead of using the word pain (which is what the patient must have felt) the word pane was used in "I've come to feel your pane". 
A glazier is someone who fixes windows, which hav

In [177]:
def generate_formatted_information_on_a_joke_for_prompt(i, jokes):
    prompt = ""
    prompt = prompt + "ID: " + jokes['ID'][i] + " \n "
    prompt = prompt + "JOKE: " + jokes['joke'][i] + " \n "
    
    explanations = ast.literal_eval(jokes['explanation'][i])
    explanations = [element for element in explanations if element]

    if len(explanations) > 0:
        prompt = prompt + "EXPLANATIONS: "
        for k in range(len(explanations)):
            if explanations[k] != '': prompt = prompt + explanations[k] + " - "

    keywords = ast.literal_eval(jokes['keywords'][i])
    keywords = list(chain.from_iterable(keywords))
    keywords = [element for element in keywords if element]
    if len(keywords) > 0:
        prompt = prompt + "KEYWORDS: "
        for k in range(len(keywords)):
            if keywords[k] != "": prompt = prompt + keywords[k] + " - "
    return prompt    





    

In [184]:
task = """Below, you can find a pun, based on the double meaning of an expression, along with it a unique identifier and, if available, some explanations and some keywords. 
I want you :
- select the expression at the core of the joke. If it is a homophone, select the two versions of it. 
- create two serious (without humor) sentences with the same expression: one where it takes the first meaning, and one where it takes the second meaning.

The result of the computation must be returned in a format like this, to fit a CSV file separated by a semicolon:
ID; expression1; expression2; sentence1; sentence2.

Note: if the expression is a omophone, write the two versions. Otherwise, just write the same expression twice.

=== PUN ===
"""



In [186]:
for i in range(len(jokes)):
    example = generate_formatted_information_on_a_joke_for_prompt(i, jokes)
    prompt = task + example
    print (talk2openai(prompt))

hom_657;dish it out;dish it out;The waitress had to dish it out to all the customers during the peak hours.;During the feedback session, she was able to dish it out to her team effectively.
het_1635;pane;pain;The glazier carefully installed the new pane in the window.;The patient experienced severe pain after the surgery.
```
hom_1510;rough;rough;He had a rough day dealing with the challenges.;The sandpaper is rough and scratches the surface.
```
```
hom_955;meet their end;meet their end;Everyone will meet their end eventually.;The contortionist could effortlessly meet their ends during the performance.
```
hom_1505;party;party;We are hosting a party at our home this Saturday.;The Liberal party announced its new policy agenda today.
```
hom_1066; breaking news; breaking news; The anchor reported some breaking news about the hurricane.; The archaeologists observed a lot of breaking news as the clay tablets began to crack.
```
hom_957;come around;come around;My cousin will come around fo